In [1]:
import torch
from torchvision.models import resnet34, resnet50
import efficientnet_pytorch

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [13]:
import sys
sys.path.append('../')
from models import DeepHomographyRegression
from ptflops import get_model_complexity_info

model = DeepHomographyRegression(shape=[3,240,320])
macs, parameters = get_model_complexity_info(model, (6, 240, 320), print_per_layer_stat=False)
print('{}: {:<30}  {:<8} parameters: {}'.format('VGG', 'FLOPS: ', float(macs.split(' ')[0])*2, float(parameters.split(' ')[0])))

VGG: FLOPS:                          11.12    parameters: 92.92


In [18]:
# install pycls as pip package
from pycls import models
import pandas as pd
from ptflops import get_model_complexity_info
import torch.nn as nn

model_names = models.model_zoo.get_model_list()
model_names
print(model_names)

df = pd.DataFrame(columns=['model', 'parameters'])

# models.model_zoo.build_model(list(model_names)[0]).stem

th = 1.e8
for name in model_names:
    model = models.model_zoo.build_model(name, pretrained=False)

    model.stem.conv = nn.Conv2d(
        in_channels=6,
        out_channels=model.stem.conv.out_channels,
        kernel_size=model.stem.conv.kernel_size,
        stride=model.stem.conv.stride,
        padding=model.stem.conv.padding
    )

    model.head.fc = nn.Linear(
        in_features=model.head.fc.in_features,
        out_features=8
    )

    macs, parameters = get_model_complexity_info(model, (6, 240, 320), print_per_layer_stat=False)
    print('{}: {:<30}  {:<8} parameters: {}'.format(name, 'FLOPS: ', float(macs.split(' ')[0])*2, float(parameters.split(' ')[0])))
    # print('{:<30}  {:<8}'.format('Number of parameters: ', parameters))
    # parameters = count_parameters(model)
    # if parameters < th:
    #     print(name, parameters)

    #     df = df.append({
    #         'model': name,
    #         'parameters': parameters
    #     }, ignore_index=True)

# for name in model_names:
#     print(name)
#     model = models.model_zoo.build_model(name, pretrained=False)
#     print(model.head.fc)
#     print(model.stem.conv)


dict_keys(['RegNetX-200MF', 'RegNetX-400MF', 'RegNetX-600MF', 'RegNetX-800MF', 'RegNetX-1.6GF', 'RegNetX-3.2GF', 'RegNetX-4.0GF', 'RegNetX-6.4GF', 'RegNetX-8.0GF', 'RegNetX-12GF', 'RegNetX-16GF', 'RegNetX-32GF', 'RegNetY-200MF', 'RegNetY-400MF', 'RegNetY-600MF', 'RegNetY-800MF', 'RegNetY-1.6GF', 'RegNetY-3.2GF', 'RegNetY-4.0GF', 'RegNetY-6.4GF', 'RegNetY-8.0GF', 'RegNetY-12GF', 'RegNetY-16GF', 'RegNetY-32GF', 'ResNet-50', 'ResNet-101', 'ResNet-152', 'ResNeXt-50', 'ResNeXt-101', 'ResNeXt-152', 'EfficientNet-B0', 'EfficientNet-B1', 'EfficientNet-B2', 'EfficientNet-B3', 'EfficientNet-B4', 'EfficientNet-B5'])
RegNetX-200MF: FLOPS:                          0.68     parameters: 2.32
RegNetX-400MF: FLOPS:                          1.32     parameters: 4.78
RegNetX-600MF: FLOPS:                          1.96     parameters: 5.67
RegNetX-800MF: FLOPS:                          2.58     parameters: 6.59
RegNetX-1.6GF: FLOPS:                          5.04     parameters: 8.29
RegNetX-3.2GF: FLOPS: 

In [16]:
import torchvision.models as models

model_names = ['resnet18', 'resnet34', 'resnet50']#, 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn']
# model_names = [model_names[3]]

# resnet 18, 34, 50 
# vgg 11, 16, 19


for name in model_names:
    model = getattr(models, name)()

    # modify in and out layers
    model.conv1 = nn.Conv2d(
        in_channels=6,
        out_channels=model.conv1.out_channels,
        kernel_size=model.conv1.kernel_size,
        stride=model.conv1.stride,
        padding=model.conv1.padding
    )
    model.fc = nn.Linear(
        in_features=model.fc.in_features,
        out_features=8
    )

    macs, parameters = get_model_complexity_info(model, (6, 240, 320), print_per_layer_stat=False)
    print('{}: {:<30}  {:<8} parameters: {}'.format(name, 'FLOPS: ', float(macs.split(' ')[0])*2, float(parameters.split(' ')[0])))    

    df = df.append({
        'model': name,
        'parameters': parameters
    }, ignore_index=True)


df.to_csv('models.csv')

resnet18: FLOPS:                          6.02     parameters: 11.19
resnet34: FLOPS:                          11.74    parameters: 21.3
resnet50: FLOPS:                          13.12    parameters: 23.53


In [24]:
import torch.nn as nn

model.features[0] = nn.Conv2d(
                    in_channels=6,
                    out_channels=model.features[0].out_channels,
                    kernel_size=model.features[0].kernel_size,
                    stride=model.features[0].stride,
                    padding=model.features[0].padding
                )

model.classifier[6] = nn.Linear(
                        in_features=model.classifier[6].in_features,
                        out_features=8
                      )

model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=8, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=8, bias=True)
)

In [3]:
# and then came the hub
repos = [
    'pytorch/vision:v0.8.2',
    'lukemelas/EfficientNet-PyTorch:master'
]

models = {}

# retrieve list
for repo in repos:
    model = torch.hub.list(repo)
    models[repo] = model

print(models)

# load model
repo = repos[1]
model = torch.hub.load(repo, models[repo][0], pretrained=False)
# model

Using cache found in /home/martin/.cache/torch/hub/pytorch_vision_v0.8.2
Using cache found in /home/martin/.cache/torch/hub/lukemelas_EfficientNet-PyTorch_master
Using cache found in /home/martin/.cache/torch/hub/facebookresearch_pycls_master


FileNotFoundError: [Errno 2] No such file or directory: '/home/martin/.cache/torch/hub/facebookresearch_pycls_master/hubconf.py'

In [101]:
# goal: add efficientnet and inception for evaluation
# inception
# weight_cpy = model.Conv2d_1a_3x3.conv.weight

# model = efficientnet_pytorch.EfficientNet.from_name('efficientnet-b0', in_channels=6)

model._conv_stem.in_features = 6
model._conv_stem.

model._fc = torch.nn.Linear(
    in_features=model._fc.in_features,
    out_features=8,
    bias=(model._fc.bias is not None)
)

# print(model._conv_stem.weight.shape)

# print(model.Conv2d_1a_3x3.conv.weight.shape)
# model.Conv2d_1a_3x3.conv = torch.nn.Conv2d(
#     in_channels=6, 
#     out_channels=model.Conv2d_1a_3x3.conv.out_channels, 
#     kernel_size=model.Conv2d_1a_3x3.conv.kernel_size, 
#     stride=model.Conv2d_1a_3x3.conv.stride, 
#     bias=(model.Conv2d_1a_3x3.conv.bias is not None)
# )
# print(model.Conv2d_1a_3x3.conv.weight.shape)

# print(model.fc.weight.shape)
# model.fc = torch.nn.Linear(
#     in_features=model.fc.in_features, 
#     out_features=8, 
#     bias=(model.fc.bias is not None)
# )
# print(model.fc.weight.shape)



In [66]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = resnet50(pretrained=False)
print(count_parameters(model))
model

25557032


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [67]:
model.fc.out_features = torch.nn.Linear(512, 8)
print(count_parameters(model))
model.fc

25561136


Linear(
  in_features=2048, out_features=Linear(in_features=512, out_features=8, bias=True), bias=True
  (out_features): Linear(in_features=512, out_features=8, bias=True)
)

In [68]:
model.conv1 = torch.nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
rnd = torch.rand([2,6,256,256])
model(rnd)
# model.conv1
model

ResNet(
  (conv1): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [69]:
eye = torch.eye(3)
rnd = torch.rand([3,3])

eye.mul(rnd)

tensor([[0.0702, 0.0000, 0.0000],
        [0.0000, 0.6360, 0.0000],
        [0.0000, 0.0000, 0.4903]])

In [70]:
rnd_inv = rnd.inverse()
rnd.matmul(rnd_inv)

eye = torch.eye(3, dtype=rnd.dtype, device=rnd.device, requires_grad=True)
eye

tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]], requires_grad=True)

In [71]:
def consistency_loss(h_ab: torch.Tensor, h_ba: torch.Tensor):
        loss = torch.nn.functional.mse_loss(h_ab.matmul(h_ba), torch.eye(3, dtype=rnd.dtype, device=rnd.device))
        return loss

rnd = torch.rand([3,3])
rnd_inv = torch.rand([3,3], requires_grad=True)

print(rnd.matmul(rnd_inv))

opt = torch.optim.Adam([rnd_inv], lr=0.1)

for _ in range(1000):
    opt.zero_grad()
    loss = consistency_loss(rnd, rnd_inv)
    loss.backward()
    opt.step()
    # print(loss)
print(rnd.matmul(rnd_inv))

tensor([[0.9629, 0.3081, 1.4567],
        [1.0948, 0.3909, 1.4366],
        [1.1110, 0.3782, 1.2908]], grad_fn=<MmBackward>)
tensor([[ 1.0000e+00,  1.2839e-07,  3.4742e-08],
        [-2.7580e-08,  1.0000e+00,  5.1945e-07],
        [ 3.0445e-08, -1.6363e-08,  1.0000e+00]], grad_fn=<MmBackward>)


In [72]:
def content_loss(f_a: torch.Tensor, f_b: torch.Tensor, m_a: torch.Tensor, m_b: torch.Tensor):
    eps = torch.finfo(f_a.dtype).eps
    loss = torch.sum(m_a.mul(m_b).mul(torch.abs(f_a.sub(f_b)))).div(torch.sum(m_a.mul(m_b) + eps))
    return loss

f_a = torch.rand([3,1,3,3])
f_b = torch.rand([3,1,3,3])
m_a = torch.eye(3)
m_a = m_a.reshape(1,1,3,3)
m_a = m_a.repeat(3,1,1,1)
m_b = m_a

eps = torch.finfo(f_a.dtype).eps
torch.sum(m_a.mul(m_b).mul(torch.abs(f_a.sub(f_b)))).div(torch.sum(m_a.mul(m_b) + eps))

tensor(0.2211)

In [73]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=0, activation: F=F.relu):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding)
        self.norm = nn.BatchNorm2d(num_features=out_channels)
        self.activation = activation

    def forward(self, x):
        x = self.conv(x)
        x = self.norm(x)
        x = self.activation(x)
        return x

conv = ConvBlock(1,3,activation=F.sigmoid)
rnd = torch.rand([3,1,3,3])

conv(rnd)

/home/martin/miniconda3/envs/hil/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor([[[[0.5969]],

         [[0.5288]],

         [[0.6166]]],


        [[[0.2022]],

         [[0.2179]],

         [[0.7142]]],


        [[[0.7272]],

         [[0.7619]],

         [[0.1992]]]], grad_fn=<SigmoidBackward>)

In [37]:
import torch
import torchvision.models as models
from pycls.models import model_zoo
from pycls.models.anynet import AnyNet

out_features = 100
hidden_size = 20
pytorch = True

# head: fc (out_features) + batch_norm -> relu
lstm = torch.nn.LSTM(
    input_size=out_features,
    hidden_size=hidden_size,
    num_layers=1
)

fc = torch.nn.Linear(
    in_features=hidden_size,
    out_features=8 # duv
)

if pytorch:
    model = models.resnet34()
    model.fc = torch.nn.Linear(
        in_features=model.fc.in_features,
        out_features=out_features
    )
else:
    model = model_zoo.build_model(name="ResNet-50")
    model.head.fc = torch.nn.Linear(
        in_features=model.head.fc.in_features,
        out_features=out_features
    )

model = torch.nn.ModuleDict({
    "encoder": model,
    "lstm_head": torch.nn.ModuleDict({
        "bn": torch.nn.BatchNorm1d(out_features),
        "relu": torch.nn.ReLU(),
        "lstm": lstm,
        "fc": fc
    })
})

model

Linear(in_features=512, out_features=100, bias=True)


ModuleDict(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_